<a href="https://colab.research.google.com/github/Abhishek3102/Huggingface-Best-Models/blob/main/Exploring_best_Huggingface_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Audio Transcription

In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 125.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

# Check for device (CUDA if available, else CPU)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Model ID
model_id = "openai/whisper-large-v3-turbo"

# Load model with appropriate dtype
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

# Load processor (feature extractor, tokenizer)
processor = AutoProcessor.from_pretrained(model_id)

# Initialize pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# Load dataset (for testing purposes, you can remove or replace this with your audio file)
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

# Path to the audio file
audio_file = "/content/Sir Alex wouldn't have done that to him Paul Scholes weighs in on the Cristiano Ronaldo situation.mp3"

# Use the pipeline to transcribe the audio, enabling return of timestamps
result = pipe(audio_file, return_timestamps=True)

# Output transcribed text (you can also access timestamps if needed)
print(result["text"])

# If you want to print out timestamps (for long-form generation)
print("Timestamps:", result.get("timestamps"))

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


 Cristiano Ronaldo has been through in the last week, Paul. You once refused to play for Manchester United in the early stages of your career, so very different to the Ronaldo situation. But for you, was it a heat of the moment decision? Yeah, you know what, it's difficult at the time to explain your emotions. Unfortunately, this week or so it's been mentioned a few times what I did and it's a low point in my career. It's something I look back on, I really regret it. It was so wrong to do. I do sympathise with Cristiano, although we know, as Owen said, he did the wrong thing. But I think my situation, we were playing Liverpool on a Sunday, and we had a game in the League Cup on the Monday night against Arsenal, which was unheard of. It was more like a reserve team game on the Monday night, and I was left out of the Sunday game against Liverpool, and I was upset about it. I wasn't very happy about it. And usually when I got left out, I understood I wasn't one to complain. but that parti

In [5]:
with open("/content/transcription_result.txt", "w") as f:
    f.write(result["text"])

# Print a confirmation message
print("Transcription saved to 'transcription_result.txt'.")

Transcription saved to 'transcription_result.txt'.


#Trancription and Sentiment Analysis Together

In [7]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# 1. Audio Transcription Setup
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

whisper_model_id = "openai/whisper-large-v3-turbo"
whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    whisper_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
whisper_model.to(device)

whisper_processor = AutoProcessor.from_pretrained(whisper_model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper_model,
    tokenizer=whisper_processor.tokenizer,
    feature_extractor=whisper_processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# 2. Sentiment Analysis Setup
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 3. Audio File Path
audio_file = "/content/Sir Alex wouldn't have done that to him Paul Scholes weighs in on the Cristiano Ronaldo situation.mp3"

# 4. Transcribe the Audio
result = pipe(audio_file, return_timestamps=True)

# 5. Get Transcribed Text
transcribed_text = result["text"]

# 6. Preprocess and Perform Sentiment Analysis
processed_text = preprocess(transcribed_text)

# Tokenize with truncation and padding
encoded_input = tokenizer(
    processed_text,
    return_tensors='pt',
    max_length=512,  # Truncate to max length
    truncation=True,  # Enable truncation
    padding=True       # Padding to make the length uniform
)

# Perform sentiment analysis
output = sentiment_model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# Print sentiment ranking
ranking = np.argsort(scores)
ranking = ranking[::-1]
sentiment_result = ""
for i in range(scores.shape[0]):
    label = config.id2label[ranking[i]]
    score = np.round(float(scores[ranking[i]]), 4)
    sentiment_result += f"{i+1}) {label} {score}\n"

# 7. Save the Transcription and Sentiment Analysis Result to a Text File
with open("/content/transcription_and_sentiment_result.txt", "w") as f:
    f.write("Transcribed Text:\n")
    f.write(transcribed_text + "\n\n")
    f.write("Sentiment Analysis Result:\n")
    f.write(sentiment_result)

# Print confirmation
print("Transcription and sentiment analysis saved to 'transcription_and_sentiment_result.txt'.")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription and sentiment analysis saved to 'transcription_and_sentiment_result.txt'.


In [9]:
!pip install transformers torch scipy nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')

# 1. Set device and load Whisper model for ASR
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# 2. Load sentiment analysis model
SENTIMENT_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_tokenizer = AutoTokenizer.from_pretrained(SENTIMENT_MODEL)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(SENTIMENT_MODEL)
sentiment_config = AutoConfig.from_pretrained(SENTIMENT_MODEL)

# 3. Preprocess function for text
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# 4. Perform ASR and transcription
audio_file_path = "/content/Sir Alex wouldn't have done that to him Paul Scholes weighs in on the Cristiano Ronaldo situation.mp3"
result = pipe(audio_file_path, return_timestamps=True)
transcribed_text = result["text"]

# 5. Split the Transcribed Text into Sentences
sentences = sent_tokenize(transcribed_text)

# 6. Initialize Result List to Store Sentence Sentiments
sentiment_results = []

# 7. Perform Sentiment Analysis for Each Sentence
for sentence in sentences:
    processed_text = preprocess(sentence)

    # Tokenize with truncation and padding
    encoded_input = sentiment_tokenizer(
        processed_text,
        return_tensors='pt',
        max_length=512,  # Truncate to max length
        truncation=True,  # Enable truncation
        padding=True       # Padding to make the length uniform
    )

    # Perform sentiment analysis
    output = sentiment_model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # Get sentiment label and score
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    sentiment = []
    for i in range(scores.shape[0]):
        label = sentiment_config.id2label[ranking[i]]
        score = np.round(float(scores[ranking[i]]), 4)
        sentiment.append(f"{i+1}) {label} {score}")

    sentiment_results.append({"sentence": sentence, "sentiment": sentiment})

# 8. Save the Transcription and Sentiment Analysis Results to a Text File
output_file_path = "/content/transcription_and_sentiment_result_per_sentence.txt"
with open(output_file_path, "w") as f:
    f.write("Transcribed Text:\n")
    f.write(transcribed_text + "\n\n")
    f.write("Sentiment Analysis Results (per sentence):\n")
    for result in sentiment_results:
        f.write(f"Sentence: {result['sentence']}\n")
        f.write("Sentiment Analysis:\n")
        for sentiment in result['sentiment']:
            f.write(f"{sentiment}\n")
        f.write("\n")

# 9. Print confirmation
print(f"Transcription and sentence-wise sentiment analysis saved to '{output_file_path}'.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/models/

Transcription and sentence-wise sentiment analysis saved to '/content/transcription_and_sentiment_result_per_sentence.txt'.
